In [ ]:
# importing the necessary libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
from tqdm import tqdm
import pandas as pd
from skimage.io import imread, imshow
from skimage.transform import resize
import scipy.ndimage
import IPython
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from tensorflow.keras import optimizers
import sklearn.model_selection
import tensorflow.keras.backend as K
%matplotlib inline

In [ ]:
base_dir = '/home/kiara/Desktop/CMR_Metric_Calculator/Dataset_Segmentation/'
train_img_dir = os.path.join(base_dir, 'PNG_images/')
train_label_dir = os.path.join(base_dir, 'PNG_labels/')

In [ ]:
# obtaining the training image (and corresponding label (masks)) file names as a list
train_img_fname = os.listdir(train_img_dir)
train_label_fname = train_img_fname

In [ ]:
# shuffling the image list randomply and saving it
train_img_fnames = random.sample(train_img_fname, len(train_img_fname))
train_label_fnames = train_img_fnames
print(len(train_label_fnames))

In [ ]:
training_dataset, test_dataset = sklearn.model_selection.train_test_split(train_img_fnames, test_size=0.1)
train_img_fnames = training_dataset
train_label_fnames = train_img_fnames

In [ ]:
test_img_fnames = test_dataset
test_label_fnames = test_img_fnames

In [ ]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNEL = 1
IMG_CHANNELS = 3

In [ ]:
len(test_img_fnames)

In [ ]:
# sanity check
print(len(train_img_fnames))

In [ ]:
# creating an array of the same dimension as the input images
X_train = np.zeros((2*len(train_img_fnames), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL), dtype = np.float32)
Y_train = np.zeros((2*len(train_img_fnames), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype = np.float32)

In [ ]:
#print("Resizing train images")
from numpy import asarray
from PIL import Image
for n, id_ in tqdm(enumerate(train_img_fnames), total=len(train_img_fnames)):
    n=n*2
    img = imread(train_img_dir + id_) # read the image
    pixels=asarray(img).astype('float32')
    pixels = resize(pixels, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL), mode = 'constant', preserve_range = True)
    # convert from integers to floats
    pixels = pixels.astype('float32')
    # normalize to the range 0-1
    pixels /= 255.0
    # confirm the normalization
    X_train[n] = pixels.astype('float32')
    
     # rotate only
    img = imread(train_img_dir + id_) # read the image
    r_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    pixels1=asarray(r_img).astype('float32')
    pixels1 = resize(pixels1, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL), mode = 'constant', preserve_range = True)
    # convert from integers to floats
    pixels1 = pixels1.astype('float32')
    # normalize to the range 0-1
    pixels1 /= 255.0
    # confirm the normalization
    X_train[n+1] = pixels1.astype('float32')
    '''
    img = imread(train_img_dir + id_) # read the image
    r_img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    pixels1=asarray(r_img).astype('float32')
    pixels1 = resize(pixels1, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL), mode = 'constant', preserve_range = True)
    # convert from integers to floats
    pixels1 = pixels1.astype('float32')
    # normalize to the range 0-1
    pixels1 /= 255.0
    # confirm the normalization
    X_train[n+2] = pixels1.astype('float32')
    
    # rotate and CLAHE
    img = cv2.imread((train_img_dir + id_), IMG_CHANNEL) # read the image
    # rotate the image
    r_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    #Converting image to LAB Color so CLAHE can be applied to the luminance channel
    lab_img= cv2.cvtColor(r_img, cv2.COLOR_BGR2LAB)
    #Splitting the LAB image to L, A and B channels, respectively
    l, a, b = cv2.split(lab_img)
    #Apply histogram equalization to the L channel
    equ = cv2.equalizeHist(l)
    #Combine the Hist. equalized L-channel back with A and B channels
    updated_lab_img1 = cv2.merge((equ,a,b))
    #Apply CLAHE to L channel
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    clahe_img = clahe.apply(l)
    #Combine the CLAHE enhanced L-channel back with A and B channels
    updated_lab_img2 = cv2.merge((clahe_img,a,b))
    #Convert LAB image back to color (RGB)
    CLAHE_img = cv2.cvtColor(updated_lab_img2, cv2.COLOR_LAB2BGR)
    pixels=asarray(CLAHE_img).astype('float32')
    pixels = resize(pixels, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL), mode = 'constant', preserve_range = True)
    # convert from integers to floats
    pixels = pixels.astype('float32')
    # normalize to the range 0-1
    pixels /= 255.0
    X_train[n+3] = pixels.astype('float32')'''
    
    
    
    
    '''# Remove comments to perform Augmentation
    #print("-----------------------CLAHE and ROTATE------------------")
    img = cv2.imread((train_img_dir + id_), IMG_CHANNEL) # read the image
    # rotate the image
    r_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    #Converting image to LAB Color so CLAHE can be applied to the luminance channel
    lab_img= cv2.cvtColor(r_img, cv2.COLOR_BGR2LAB)
    #Splitting the LAB image to L, A and B channels, respectively
    l, a, b = cv2.split(lab_img)
    #Apply histogram equalization to the L channel
    equ = cv2.equalizeHist(l)
    #Combine the Hist. equalized L-channel back with A and B channels
    updated_lab_img1 = cv2.merge((equ,a,b))
    #Apply CLAHE to L channel
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    clahe_img = clahe.apply(l)
    #Combine the CLAHE enhanced L-channel back with A and B channels
    updated_lab_img2 = cv2.merge((clahe_img,a,b))
    #Convert LAB image back to color (RGB)
    CLAHE_img = cv2.cvtColor(updated_lab_img2, cv2.COLOR_LAB2BGR)
    pixels=asarray(CLAHE_img).astype('float32')
    pixels = resize(pixels, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL), mode = 'constant', preserve_range = True)
    # convert from integers to floats
    pixels = pixels.astype('float32')
    # normalize to the range 0-1
    pixels /= 255.0
    X_train[n+3] = pixels.astype('float32')'''

    '''
    #print("-----------------------CLAHE ONLY ------------------")
    img = cv2.imread((train_img_dir + id_), IMG_CHANNEL) # read the image
    # rotate the image
    r_img = img
    #Converting image to LAB Color so CLAHE can be applied to the luminance channel
    lab_img= cv2.cvtColor(r_img, cv2.COLOR_BGR2LAB)
    #Splitting the LAB image to L, A and B channels, respectively
    l, a, b = cv2.split(lab_img)
    #Apply histogram equalization to the L channel
    equ = cv2.equalizeHist(l)
    #Combine the Hist. equalized L-channel back with A and B channels
    updated_lab_img1 = cv2.merge((equ,a,b))
    #Apply CLAHE to L channel
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    clahe_img = clahe.apply(l)
    #Combine the CLAHE enhanced L-channel back with A and B channels
    updated_lab_img2 = cv2.merge((clahe_img,a,b))
    #Convert LAB image back to color (RGB)
    CLAHE_img = cv2.cvtColor(updated_lab_img2, cv2.COLOR_LAB2BGR)
    pixels=asarray(CLAHE_img).astype('float32')
    pixels = resize(pixels, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL), mode = 'constant', preserve_range = True)
    # convert from integers to floats
    pixels = pixels.astype('float32')
    # normalize to the range 0-1
    pixels /= 255.0
    X_train[n+2] = pixels.astype('float32')
    '''
    '''#print("-----------------------CLAHE AND ROTATE COUNTER ONLY------------------")
    img = cv2.imread((train_img_dir + id_), 1) # read the image
    # rotate the image
    r_img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    #Converting image to LAB Color so CLAHE can be applied to the luminance channel
    lab_img= cv2.cvtColor(r_img, cv2.COLOR_BGR2LAB)
    #Splitting the LAB image to L, A and B channels, respectively
    l, a, b = cv2.split(lab_img)
    #Apply histogram equalization to the L channel
    equ = cv2.equalizeHist(l)
    #Combine the Hist. equalized L-channel back with A and B channels
    updated_lab_img1 = cv2.merge((equ,a,b))
    #Apply CLAHE to L channel
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    clahe_img = clahe.apply(l)
    #Combine the CLAHE enhanced L-channel back with A and B channels
    updated_lab_img2 = cv2.merge((clahe_img,a,b))
    #Convert LAB image back to color (RGB)
    CLAHE_img = cv2.cvtColor(updated_lab_img2, cv2.COLOR_LAB2BGR)
    pixels=asarray(CLAHE_img).astype('float32')
    pixels = resize(pixels, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL), mode = 'constant', preserve_range = True)
    # convert from integers to floats
    pixels = pixels.astype('float32')
    # normalize to the range 0-1
    pixels /= 255.0
    X_train[n+2] = pixels.astype('float32')'''
    

In [ ]:
print("Resizing train images")
from numpy import asarray
from PIL import Image
for n, id_ in tqdm(enumerate(train_img_fnames), total=len(train_img_fnames)):
    n=n*2
    img = imread(train_label_dir + id_) # read the image
    pixels=asarray(img).astype('float32')
    pixels = resize(pixels, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), mode = 'constant', preserve_range = True)
    # convert from integers to floats
    pixels = pixels.astype('float32')
    # normalize to the range 0-1
    pixels /= 255.0
    Y_train[n] = pixels.astype('float32')
    
     #clahe and rotate
    img = imread(train_label_dir + id_) # read the image
    r_img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    pixels1=asarray(r_img).astype('float32')
    pixels1 = resize(pixels1, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), mode = 'constant', preserve_range = True)
    # convert from integers to floats
    pixels1 = pixels1.astype('float32')
    # normalize to the range 0-1
    pixels1 /= 255.0
    # confirm the normalization
    Y_train[n+1] = pixels1.astype('float32')
    '''
    # clahe and rotate counter
    img = imread(train_label_dir + id_) # read the image
    r_img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    pixels1=asarray(r_img).astype('float32')
    pixels1 = resize(pixels1, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), mode = 'constant', preserve_range = True)
    # convert from integers to floats
    pixels1 = pixels1.astype('float32')
    # normalize to the range 0-1
    pixels1 /= 255.0
    # confirm the normalization
    Y_train[n+2] = pixels1.astype('float32')
    
    Y_train[n+3] = Y_train[n+1]'''
    '''
    # clahe only
    #Y_train[n+2] = Y_train[n]'''
    
    '''# clahe and rotate counter
    img = imread(train_label_dir + id_) # read the image
    r_img = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
    pixels1=asarray(r_img).astype('float32')
    pixels1 = resize(pixels1, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), mode = 'constant', preserve_range = True)
    # convert from integers to floats
    pixels1 = pixels1.astype('float32')
    # normalize to the range 0-1
    pixels1 /= 255.0
    # confirm the normalization
    Y_train[n+2] = pixels1.astype('float32')'''
    
    #Y_train[n+3] = Y_train[n+1]

In [ ]:
# plotting an image
seed = 17
np.random.seed = seed
image_x = random.randint(0, len(train_img_fnames)) # generate a random number between 0 and length of training ids
imshow(np.squeeze(X_train[image_x]))
#plt.savefig("image.pdf", format='pdf')
plt.show()

In [ ]:
imshow(np.squeeze(Y_train[image_x]))
#plt.savefig("label.pdf", format='pdf')
plt.show()

In [ ]:
X_test = np.zeros((len(test_img_fnames), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL), dtype = np.float32)
sizes_test = []
print("Resizing test images")
for n, id_ in tqdm(enumerate(test_img_fnames), total=len(test_img_fnames)):
    path = base_dir
    img = imread(train_img_dir + id_) # read the image
    # Uncomment to test on HELIX Dataset
    #img = imread('/media/kiara/My Passport/HELIX/image/' + id_)
    pixels=asarray(img).astype('float32')
    pixels = resize(pixels, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNEL), mode = 'constant', preserve_range = True)
    
    # convert from integers to floats
    pixels = pixels.astype('float32')
    
    # normalize to the range 0-1
    pixels /= 255.0
    X_test[n] = pixels.astype('float32')

In [ ]:
Y_test = np.zeros((len(test_label_fnames), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype = np.float32)
print("Resizing test images")
from numpy import asarray
from PIL import Image
for n, id_ in tqdm(enumerate(test_img_fnames), total=len(test_img_fnames)):
    #path = base_dir
    img = imread(train_label_dir + id_) # read the image
    #img = imread('/media/kiara/My Passport/HELIX/label/ShortAxis/' + id_)
    pixels=asarray(img).astype('float32')
    pixels = resize(pixels, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), mode = 'constant', preserve_range = True)
    
    # convert from integers to floats
    pixels = pixels.astype('float32')
    
    # normalize to the range 0-1
    pixels /= 255.0
    Y_test[n] = pixels.astype('float32')

In [ ]:
seed = 17
np.random.seed = seed
image_x = random.randint(0, len(test_img_fnames)) # generate a random number between 0 and length of training ids
imshow(np.squeeze(X_test[image_x]))
#plt.savefig("image.pdf", format='pdf')
plt.show()

In [ ]:
imshow(np.squeeze(Y_test[image_x]))
#plt.savefig("label.pdf", format='pdf')
plt.show()

## Metrics and Function Definitions

In [ ]:
def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)
    return iou

def precision(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [ ]:
def DC(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-DC(y_true, y_pred)

In [ ]:
# importing packages necessary for model training
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import random
from tqdm import tqdm
import pandas as pd
from skimage.io import imread, imshow
from skimage.transform import resize
import scipy.ndimage
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
import IPython
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from numpy import asarray
from PIL import Image
import sklearn.model_selection

## Model

In [ ]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNEL = 1
IMG_CHANNELS = 3

In [ ]:
# defining input layer
inputs = tf.keras.layers.Input((IMG_WIDTH, IMG_HEIGHT, IMG_CHANNEL))
# pixels to floating point numbers
s = tf.keras.layers.Lambda(lambda x: (x/255))(inputs)
print(s)

# MODEL 1 (Unet with Dropout Layers)

In [ ]:
c1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.5)(c1)
c1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.5)(c2)
c2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.5)(c3)
c3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.5)(c4)
c4 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.5)(c5)
c5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
c5 = tf.keras.layers.Dropout(0.5)(c5)

# Expansion Path
u6 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.5)(c6)
c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.5)(c7)
c7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.5)(c8)
c8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.5)(c9)
c9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

outputs = tf.keras.layers.Conv2D(3, (1,1), activation='sigmoid')(c9)

In [ ]:
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
opt = keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics=[DC, iou_coef, 'acc', precision, recall])
model.summary()

In [ ]:
# model checkpoint
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='/home/kiara/Desktop/CMR_Metric_Calculator/UNET_WITH_2000RERUN.h5', verbose = 2, save_weights_only = True)
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience = 50, monitor = 'val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir = 'Log_UNET_WITH_2000RERUN')
]
# change name to 2000RERUNAug

# FIT MODEL
results = model.fit(X_train, Y_train, validation_split = 0.1, batch_size = 4, epochs = 150, callbacks=callbacks)

model.save('model_UNET_WITH_2000RERUN')
model.save('model_UNET_WITH_2000RERUN.h5')  

In [ ]:
model.evaluate(X_test, Y_test, verbose=1)

# MODEL 2 (Unet without Dropout Layers)

In [ ]:
#Contraction path
c1 = tf.keras.layers.Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(s)
#c1 = tf.keras.layers.Dropout(0.5)(c1)
c1 = tf.keras.layers.Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p1)
#c2 = tf.keras.layers.Dropout(0.5)(c2)
c2 = tf.keras.layers.Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p2)
#c3 = tf.keras.layers.Dropout(0.4)(c3)
c3 = tf.keras.layers.Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p3)
#c4 = tf.keras.layers.Dropout(0.5)(c4)
c4 = tf.keras.layers.Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(512, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(p4)
#c5 = tf.keras.layers.Dropout(0.4)(c5)
c5 = tf.keras.layers.Conv2D(512, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c5)
#c5 = tf.keras.layers.Dropout(0.5)(c5)

# Expansion Path
u6 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(256, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(u6)
#c6 = tf.keras.layers.Dropout(0.4)(c6)
c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(u7)
#c7 = tf.keras.layers.Dropout(0.5)(c7)
c7 = tf.keras.layers.Conv2D(128, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(u8)
#c8 = tf.keras.layers.Dropout(0.5)(c8)
c8 = tf.keras.layers.Conv2D(64, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(u9)
#c9 = tf.keras.layers.Dropout(0.4)(c9)
c9 = tf.keras.layers.Conv2D(32, (3, 3), activation='tanh', kernel_initializer='he_normal', padding='same')(c9)

outputs = tf.keras.layers.Conv2D(3, (1,1), activation='sigmoid')(c9)

In [ ]:
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
opt = keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics=[DC, iou_coef, 'acc', precision, recall])
model.summary()

In [ ]:
# model checkpoint
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='/home/kiara/Desktop/CMR_Metric_Calculator/UNET_WITHOUT_10000AugRERUN.h5', verbose = 2, save_weights_only = True)
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience = 50, monitor = 'val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir = 'Log_UNET_WITHOUT_10000AugRERUN')
]


# FIT MODEL
results = model.fit(X_train, Y_train, validation_split = 0.1, batch_size = 4, epochs = 150, callbacks=callbacks)

model.save('model_UNET_WITHOUT_10000AugRERUN')
model.save('model_UNET_WITHOUT_10000AugRERUN.h5')  

In [ ]:
model.evaluate(X_test, Y_test, verbose=1)

# MODEL 3 (ResUnet)

In [ ]:
# taken directly from the original implementation https://arxiv.org/pdf/1711.10684.pdf
def bn_act(x, act=True):
    x = tf.keras.layers.BatchNormalization()(x)
    if act == True:
        x = tf.keras.layers.Activation("tanh")(x)
        #x = tf.keras.layers.Activation("sigmoid")(x)
    return x

def conv_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    conv = bn_act(x)
    conv = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(conv)
    return conv

def stem(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    conv = tf.keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    conv = conv_block(conv, filters, kernel_size=kernel_size, padding=padding, strides=strides)
    
    shortcut = tf.keras.layers.Conv2D(filters, kernel_size=(1, 1), padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)
    
    output = tf.keras.layers.Add()([conv, shortcut])
    return output

def residual_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):
    res = conv_block(x, filters, kernel_size=kernel_size, padding=padding, strides=strides)
    res = conv_block(res, filters, kernel_size=kernel_size, padding=padding, strides=1)
    
    shortcut = tf.keras.layers.Conv2D(filters, kernel_size=(1, 1), padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)
    
    output = tf.keras.layers.Add()([shortcut, res])
    return output

def upsample_concat_block(x, xskip):
    u = tf.keras.layers.UpSampling2D((2, 2))(x)
    c = tf.keras.layers.concatenate([u, xskip])
    return c

In [ ]:
# taken directly from the original implementation https://arxiv.org/pdf/1711.10684.pdf
def ResUNet():
    
    #f = [24, 48, 96, 192, 384]
    #f = [8, 16, 32, 64, 128]
    #f = [64, 128, 256, 512, 1024]
    #f = [8, 16, 32, 64, 128]#1st
    # 16 2nd then 4 3rd
    f = [4, 8, 16, 32, 64]
    
    #f = [32, 64, 128, 256, 512]
    inputs = keras.layers.Input((256, 256, 1))
    
    ## Encoder
    e0 = inputs
    e1 = stem(e0, f[0])
    e2 = residual_block(e1, f[1], strides=2)
    e3 = residual_block(e2, f[2], strides=2)
    e4 = residual_block(e3, f[3], strides=2)
    e5 = residual_block(e4, f[4], strides=2)
    
    ## Bridge
    b0 = conv_block(e5, f[4], strides=1)
    b1 = conv_block(b0, f[4], strides=1)
    
    ## Decoder
    u1 = upsample_concat_block(b1, e4)
    d1 = residual_block(u1, f[4])
    
    u2 = upsample_concat_block(d1, e3)
    d2 = residual_block(u2, f[3])
    
    u3 = upsample_concat_block(d2, e2)
    d3 = residual_block(u3, f[2])
    
    u4 = upsample_concat_block(d3, e1)
    d4 = residual_block(u4, f[1])
    
    outputs = keras.layers.Conv2D(3, (1, 1), padding="same", activation="sigmoid")(d4)
    model = keras.models.Model(inputs, outputs)
    return model

In [ ]:
model = ResUNet()
from keras.utils import to_categorical
opt = keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics=[DC, iou_coef, 'acc', precision, recall])
model.summary()

In [ ]:
# model checkpoint
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='/home/kiara/Desktop/CMR_Metric_Calculator/RESUNET_10000.h5', verbose = 2, save_weights_only = True)
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience = 50, monitor = 'val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir = 'Log_RESUNET_10000')
]


# FIT MODEL
results = model.fit(X_train, Y_train, validation_split = 0.1, batch_size = 4, epochs = 150, callbacks=callbacks)

model.save('model_RESUNET_10000')
model.save('model_RESUNET_10000.h5')  

In [ ]:
model.evaluate(X_test, Y_test, verbose=1)

# MODEL 4 (FCN)

In [ ]:
c1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.5)(c1)
c1 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.5)(c2)
c2 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.5)(c3)
c3 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.5)(c4)
c4 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.5)(c5)
c5 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)
c5 = tf.keras.layers.Dropout(0.5)(c5)

u6 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c5)
c6 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.5)(c6)
c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c6)
c7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.5)(c7)
c7 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.5)(c8)
c8 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.5)(c9)
c9 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

# In[13]:


outputs = tf.keras.layers.Conv2D(3, (1,1), activation='sigmoid')(c9)

In [ ]:
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
opt = keras.optimizers.Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics=[DC, iou_coef, 'acc', precision, recall])
model.summary()

In [ ]:
# model checkpoint
checkpointer = tf.keras.callbacks.ModelCheckpoint(filepath='/home/kiara/Desktop/CMR_Metric_Calculator/FCN_10000.h5', verbose = 2, save_weights_only = True)
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience = 50, monitor = 'val_loss'),
    tf.keras.callbacks.TensorBoard(log_dir = 'Log_FCN_10000')
]


# FIT MODEL
results = model.fit(X_train, Y_train, validation_split = 0.1, batch_size = 4, epochs = 150, callbacks=callbacks)

model.save('model_FCN_10000')
model.save('model_FCN_10000.h5')  

In [ ]:
model.evaluate(X_test, Y_test, verbose=1)